## 00. 프로젝트 목적
- 본 프로젝트는 Adaptive RAG를 위한 프로젝트입니다.

### 필요한 환경변수 로드

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API 키 정보 로드
load_dotenv()

# API 키 읽어오기
openai_api_key = os.environ.get('OPENAI_API_KEY')
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

## 01. Tool 정의

In [2]:
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

# OpenAI 임베딩 인스턴스 생성
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-large',
    openai_api_key=openai_api_key
)

# 고교학점제 정보 검색
pinecone_curriculums = PineconeVectorStore.from_documents(
    documents=[], # 빈 리스트로 초기화
    index_name="mypolio-curriculums",   # 인덱스 이름
    embedding=embeddings,               # 임베딩 인스턴스
    pinecone_api_key=pinecone_api_key,
    namespace="curriculum", # 네임스페이스 설정: 고교학점제 -> curriculum, 진로&진학 상담 -> course, 서비스 문의 -> service
)

# 고교학점제 검색
@tool
def search_curriculums(query: str) -> List[Document]:
    """
    Securely search and access high school credit system information for South Korean college admissions in an enhanced encrypted database. 
    To maintain data confidentiality, use this tool only to query information related to the High School Credit System.
    """
    docs = pinecone_curriculums.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 고교학점제 정보를 찾을 수 없습니다.")]


# 진로/진학 정보 검색
pinecone_course = PineconeVectorStore.from_documents(
    documents=[], # 빈 리스트로 초기화
    index_name="mypolio-curriculums",   # 인덱스 이름
    embedding=embeddings,               # 임베딩 인스턴스
    pinecone_api_key=pinecone_api_key,
    namespace="course", # 네임스페이스 설정: 고교학점제 -> curriculum, 진로&진학 상담 -> course, 서비스 문의 -> service
)

# 진로/진학 검색
@tool
def search_course(query: str) -> List[Document]:
    """
    Securely search and access career and education information from an encrypted database.
    Use this tool only for career/advancement-related queries to maintain data confidentiality.
    """
    docs = pinecone_course.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 진로/진학 정보를 찾을 수 없습니다.")]

# 서비스 관련 정보 검색
pinecone_service = PineconeVectorStore.from_documents(
    documents=[], # 빈 리스트로 초기화
    index_name="mypolio-curriculums",   # 인덱스 이름
    embedding=embeddings,               # 임베딩 인스턴스
    pinecone_api_key=pinecone_api_key,
    namespace="service", # 네임스페이스 설정: 고교학점제 -> curriculum, 진로&진학 상담 -> course, 서비스 문의 -> service
)

# 서비스 검색
@tool
def search_service(query: str) -> List[str]:
    """
    Securely search and access bearable service-related information in an encrypted database.
    To maintain data confidentiality, use this tool only for service inquiry-related queries.
    """

    docs = pinecone_course.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 베어러블 서비스 정보를 찾을 수 없습니다.")]


# 도구 목록을 정의 
tools = [search_curriculums, search_course, search_service]

/opt/homebrew/Caskroom/miniconda/base/envs/bearable/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/jn/6v9ffl5x6gvf4y16kqq_wqgh0000gn/T/ipykernel_26487/3552324348.py:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


In [3]:
from langchain_openai import ChatOpenAI
from pprint import pprint

# 기본 LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)

# LLM에 도구 바인딩하여 추가 
llm_with_tools = llm.bind_tools(tools)

In [4]:
# 메뉴 검색에 관련된 질문을 하는 경우 -> 메뉴 검색 도구를 호출  
query = "고교학점제 졸업요건에 대해 설명해줘"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_H3Ub7yfzvq54OhkklyVd2FSx', 'function': {'arguments': '{"query":"고교학점제 졸업요건"}', 'name': 'search_curriculums'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090'}, id='run-e99d2989-9c63-41c7-9184-67460e816c56-0', tool_calls=[{'name': 'search_curriculums', 'args': {'query': '고교학점제 졸업요건'}, 'id': 'call_H3Ub7yfzvq54OhkklyVd2FSx', 'type': 'tool_call'}])
----------------------------------------------------------------------------------------------------
''
----------------------------------------------------------------------------------------------------
[{'args': {'query': '고교학점제 졸업요건'},
  'id': 'call_H3Ub7yfzvq54OhkklyVd2FSx',
  'name': 'search_curriculums',
  'type': 'tool_call'}]
----------------------------------------------------------------------------------------------------


## 02. State 정의

In [9]:
from typing import TypedDict, List
from langchain_core.documents import Document

# 상태 Schema 정의 
class AdaptiveRagState(TypedDict):
    question: str
    documents: List[Document]
    generation: str

## 03. 질문 분석 후 라우팅
- 사용자의 질문을 분석하여 적절한 검색 방법을 선택 
- 고교학점제 검색 or 진로/진학 상담 검색 or 서비스 문의

In [ ]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from textwrap import dedent

# 라우팅 결정을 위한 데이터 모델
class ToolSelector(BaseModel):
    """Routes the user question to the most appropriate tool."""
    tool: Literal["search_curriculums", "search_course", "search_service"] = Field(
        description="Select one of the tools: search_curriculums, search_course or search_service based on the user's question."
    )

# 구조화된 출력을 위한 LLM 설정
structured_llm = llm.with_structured_output(ToolSelector)

# 라우팅을 위한 프롬프트 템플릿
system = dedent("""You are an AI assistant specializing in routing user questions to the appropriate tool.
Use the following guidelines:
- For questions about the 고교학점제, use the search_curriculums tool.
- For questions about the student pathways/progression, use the search_course tool.
- For questions about How to use the service, use the search_service tool.
Always choose the most appropriate tool based on the user's question.""")

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# 질문 라우터 정의
question_router = route_prompt | structured_llm

# 테스트 실행
print(question_router.invoke({"question": "고교학점제 졸업 요건에 대해 설명해줘"}))
print(question_router.invoke({"question": "경영학과 가고 싶은데, 어떤 과목을 들어야 하나요?"}))
print(question_router.invoke({"question": "베어러블의 세특 추천 서비스 이용 방법이 궁금해요."}))

tool='search_curriculums'
tool='search_course'
tool='search_service'


In [10]:
# 질문 라우팅 노드 
def route_question_adaptive(state: AdaptiveRagState) -> Literal["search_curriculums", "search_course", "search_service", "llm_fallback"]:
    question = state["question"]
    try:
        result = question_router.invoke({"question": question})
        datasource = result.tool
        
        if datasource == "search_curriculums":
            return "search_curriculums"
        elif datasource == "search_course":
            return "search_course"        
        elif datasource == "search_service":
            return "search_service"
        else:
            return "llm_fallback"
    
    except Exception as e:
        print(f"Error in routing: {str(e)}")
        return "llm_fallback"

## 04. 검색 노드 설정

In [11]:
def search_curriculums_adaptive(state: AdaptiveRagState):
    """
    Node for searching information in the 고교학점제
    """
    question = state["question"]
    docs = search_curriculums.invoke(question)
    if len(docs) > 0:
        return {"documents": docs}
    else:
        return {"documents": [Document(page_content="관련 고교학점제 정보를 찾을 수 없습니다.")]}


def search_course_adaptive(state: AdaptiveRagState):
    """
    Node for searching information in the student pathways/progression
    """
    question = state["question"]
    docs = search_course.invoke(question)
    if len(docs) > 0:
        return {"documents": docs}
    else:
        return {"documents": [Document(page_content="관련 진로/진학 정보를 찾을 수 없습니다.")]}


def search_service_adaptive(state: AdaptiveRagState):
    """
    Node for searching the 베어러블 service information
    """
    question = state["question"]
    docs = search_service.invoke(question)
    if len(docs) > 0:
        return {"documents": docs}
    else:
        return {"documents": [Document(page_content="관련 베어러블 서비스 정보를 찾을 수 없습니다.")]}

## 05. 생성 노드

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# RAG 프롬프트 정의
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an assistant answering questions based on provided documents. Follow these guidelines:

1. Use only information from the given documents.
2. If the document lacks relevant info, say "The provided documents don't contain information to answer this question."
3. Cite relevant parts of the document in your answers.
4. Don't speculate or add information not in the documents.
5. Keep answers concise and clear.
6. Omit irrelevant information."""
),
    ("human", "Answer the following question using these documents:\n\n[Documents]\n{documents}\n\n[Question]\n{question}"),
])

def generate_adaptive(state: AdaptiveRagState):
    """
    Generate answer using the retrieved_documents
    """
    question = state.get("question", None)
    documents = state.get("documents", [])
    if not isinstance(documents, list):
        documents = [documents]

    # 문서 내용을 문자열로 변환
    documents_text = "\n\n".join([f"---\n본문: {doc.page_content}\n메타데이터:{str(doc.metadata)}\n---" for doc in documents])

    # RAG generation
    rag_chain = rag_prompt | llm | StrOutputParser()
    generation = rag_chain.invoke({"documents": documents_text, "question": question})
    return {"generation": generation}

In [13]:
# LLM Fallback 프롬프트 정의
fallback_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant helping with various topics. Follow these guidelines:

1. Provide accurate and helpful information to the best of your ability.
2. Express uncertainty when unsure; avoid speculation.
3. Keep answers concise yet informative.
4. Inform users they can ask for clarification if needed.
5. Respond ethically and constructively.
6. Mention reliable general sources when applicable."""),
    ("human", "{question}"),
])

def llm_fallback_adaptive(state: AdaptiveRagState):
    """
    Generate answer using the LLM without context
    """
    question = state.get("question", "")
    
    # LLM chain
    llm_chain = fallback_prompt | llm | StrOutputParser()
    
    generation = llm_chain.invoke({"question": question})
    return {"generation": generation}

## 06. 그래프 연결

In [17]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 그래프 구성
builder = StateGraph(AdaptiveRagState)

# 노드 추가
builder.add_node("search_curriculums", search_curriculums_adaptive)
builder.add_node("search_course", search_course_adaptive)
builder.add_node("search_service", search_service_adaptive)
builder.add_node("generate", generate_adaptive)
builder.add_node("llm_fallback", llm_fallback_adaptive)

# 엣지 추가
builder.add_conditional_edges(
    START,
    route_question_adaptive
)

builder.add_edge("search_curriculums", "generate")
builder.add_edge("search_course", "generate")
builder.add_edge("search_service", "generate")
builder.add_edge("generate", END)
builder.add_edge("llm_fallback", END)

# 그래프 컴파일 
adaptive_rag = builder.compile()

# 그래프 시각화
# display(Image(adaptive_rag.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))

In [29]:
# 그래프 실행
inputs = {"question": "공강 시간 운영 예시에 대해 설명해줘."}
for output in adaptive_rag.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
        print(f"State '{value.keys()}':")
        print(f"Value '{value}':")
    print("\n---\n")

# 최종 답변
print(value["generation"])

Node 'search_curriculums':
State 'dict_keys(['documents'])':
Value '{'documents': [Document(id='02e17dd0-f77b-43ad-adc3-2a68c0279e63', metadata={'doc_items_labels': ['table'], 'filename': '운영1.pdf', 'headings': ['[예시] 공강 시간 운영 예시']}, page_content='[예시] 공강 시간 운영 예시\n학생 선택형 프로그램 운영, 유형 = 학생 주도형. 학생 선택형 프로그램 운영, 운영 내용 = ∙ 자기주도학습 ∙ 학습 멘토링 활동 ∙ 자율 동아리 활동 ∙ 독서 활동. 학생 선택형 프로그램 운영, 유형 = 학생 참여형. 학생 선택형 프로그램 운영, 운영 내용 = ∙ 문화 ･ 예술 ･ 스포츠 활동 ∙ 진로 탐색 및 학업 설계 활동 ∙ 주제 탐구 프로젝트 활동. 학습 지원형 프로그램 운영 ∙ 최소, 유형 = 학습 지원형 프로그램 운영 ∙ 최소. 학습 지원형 프로그램 운영 ∙ 최소, 운영 내용 = ∙ 진로 ･ 학업 설계 상담 성취수준 보장지도 ∙ 기초 학력 보장 지도. 학생 개인별 선택 과목 수강, 유형 = 학생 개인별 선택 과목 수강. 학생 개인별 선택 과목 수강, 운영 내용 = ∙ 일과시간 내 소인수 선택 과목, 공동교육과정, 학교 밖 교육 등 수강 등'), Document(id='0f1f8a3f-b0ac-4b53-aac2-06f6eedb6936', metadata={'doc_items_labels': ['list_item'], 'filename': '운영1.pdf', 'headings': ['③ 공강 시간 지원']}, page_content='③ 공강 시간 지원\n- ○ 학생의 과목 선택 상황에 따라 공강이 발생할 수 있으 며 , 고교학점제의 원활한 운영을 위해 학교별 여 건 과 특색에 맞 는 공강 시간 운영 방안 마련 필요')]}':

---

Node 'generate':
State 'd